# Wiederholung: SQL II

In [ ]:
%CREATE :memory:

## Inhaltsverzeichnis
- [CSV-Import](#CSV-Import)
- [Kreuztabellen](#Kreuztabellen)
- [Cube](#Cube)
- [Rollup](#Rollup)
- [Grouping Sets](#Grouping-Sets)
- [Fensterfunktionen](#Fensterfunktionen)

## CSV-Import
**Achtung**: Das Laden von Dateien ist abhängig vom verwendeten Datenbanksystem. Bitte beachten Sie die Hilfeseiten zu dem von Ihnen eingesetzten DBMS!

Die folgende Anfrage erzeugt eine View mit dem Inhalt der CSV-Datei.

In [ ]:
CREATE OR REPLACE VIEW verkauf AS
FROM read_csv(verkauf.csv, header=true, delim=',', auto_detect=true)

In [ ]:
%MAX_ROWS 10
SELECT * FROM verkauf

Bei jeder Anfrage wird die CSV-Datei neu eingelesen. Sie können das prüfen, indem Sie parallel die Datei mit einem Texteditor verändern und dann nur das zuletzt erfolgte `SELECT`-Statement neu ausführen, ohne die zuvor erzeugte View neuzuladen.

## Kreuztabellen

In [ ]:
PIVOT verkauf
-- Attribut + Beschränkung für Spalten
ON Bundesland IN ("Thüringen", "Sachsen-Anhalt")
-- Aggregation für Werte
USING SUM(Volumen)
-- Attribut für Zeilen
GROUP BY Jahr
ORDER BY Jahr

## Cube

In [ ]:
%ALL_ROWS
SELECT Jahr, Bundesland, PGruppe, SUM(Volumen)
FROM verkauf
--
WHERE Jahr IN (2021, 2022) AND Bundesland IN ('Thüringen', 'Sachsen-Anhalt')
--
GROUP BY CUBE(Jahr, Bundesland, PGruppe)

## Rollup

In [ ]:
%ALL_ROWS
SELECT Jahr, Bundesland, SUM(Volumen)
FROM verkauf
--
WHERE Jahr IN (2021, 2022) AND Bundesland IN ('Thüringen', 'Sachsen-Anhalt')
--
GROUP BY ROLLUP(Jahr, Bundesland)

In [ ]:
%ALL_ROWS
SELECT Bundesland, Jahr, SUM(Volumen)
FROM verkauf
--
WHERE Jahr IN (2021, 2022) AND Bundesland IN ('Thüringen', 'Sachsen-Anhalt')
--
GROUP BY ROLLUP(Bundesland, Jahr)

## Grouping Sets

In [ ]:
%ALL_ROWS
SELECT Bundesland, Jahr, SUM(Volumen)
FROM verkauf
--
WHERE Jahr IN (2021, 2022) AND Bundesland IN ('Thüringen', 'Sachsen-Anhalt')
--
GROUP BY GROUPING SETS((Jahr), (Bundesland))

## Fensterfunktionen

In [ ]:
%ALL_ROWS
SELECT Bundesland, Volumen, RANK() OVER (ORDER BY Volumen DESC)
FROM verkauf
WHERE Jahr = 2023 AND PGruppe = 'Wein'

In [ ]:
%ALL_ROWS
SELECT Bundesland, PGruppe, Volumen, RANK() OVER (PARTITION BY PGruppe ORDER BY Volumen DESC)
FROM verkauf
WHERE Jahr = 2023

In [ ]:
%ALL_ROWS
SELECT
    *,
    AVG(Volumen) OVER(PARTITION BY Bundesland ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING)::int AS GM3
FROM verkauf
WHERE PGruppe = 'Wein' AND Bundesland = 'Thüringen'